In [ ]:
!pip install googletrans==3.1.0a0
import pandas as pd
import re
from googletrans import Translator

# from google.colab import drive
# drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.4 MB 15.7 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 4.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=34f657515ebe3d2b54cb0ab3ea48ca9cf76a5009906a0b96bc0ba5f092de8b3a
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning on practice/NLP/train.csv', index_col = 'pair_id')
df

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...
497815,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497816,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497817,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0


In [ ]:
df_0 = df[df["is_duplicate"] == 0]
df_0

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0
...,...,...,...
497815,BIT-MAT PRODUCTS,The Goodyear Tire and Rubber Company,0
497816,"Bnd Trading Co., Ltd.",Zhong Shan Yue Liang Economy& Trade Imp. & Exp...,0
497817,"Xeikon Industrial Co., Ltd. Of Dongguan City","Yi Cheng Trading Co., Ltd. Of Dongguan City",0


In [ ]:
df_1 = df[df["is_duplicate"] == 1]
df_1

,name_1,name_2,is_duplicate
pair_id,,,
162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
...,...,...,...
496575,"Bridgestone （Huizhou）Synthetic Rubber Co., Ltd.","Bridgestone India Pvt., Ltd.",1
496761,Arlanxeo International Group,Arlanxeo Corp.,1
497016,Brenntag Peru S.A.C.,Brenntag Chile Comercial E Industrial Ltda,1


In [ ]:
filter_words = {
    'co', 'cor', 'corp', 'corporation',
    'ltd', 'llp', 'llc', 'ltda', 'inc', 'plc', 'group', 'ооо', 'зао', 'оао',
    'canada', 'europe', 'japan', 'russia', 'rumunia'
    'polska', 'czech', 'france', 'peruana', 'mexico'
    'spain', 'brasil', 'singapore', 'india', 
    'usa', 'vietnam', 'argentina', 'slovakia'
}

translator = Translator()

def compare(df):
    copy = df.copy()
    for index, row in copy.iterrows():
        name_1 = re.sub('["\.,«»&:)(\'-]', ' ', row["name_1"].lower())
        name_2 = re.sub('["\.,«»&:)(\'-]', ' ', row["name_2"].lower())
        
        words_1 = [ translator.translate(word, dest='en').text.lower()
                        for word in re.split('\s+', name_1) if word != '' and word not in filter_words]
        words_2 = [ translator.translate(word, dest='en').text.lower()
                        for word in re.split('\s+', name_2) if word != '' and word not in filter_words]
        
        for word_1 in words_1:
            is_duplicate = False
            for word_2 in words_2:
                if word_1 == word_2:
                    copy.at[index, 'is_duplicate'] = 1
                    is_duplicate = True
                    break
            if is_duplicate:
                break

        if not is_duplicate:
            copy.at[index, 'is_duplicate'] = 0

    return copy

In [ ]:
duplicates = compare(df_1.sample(1000))
differences = compare(df_0.sample(1000))

In [ ]:
accuracy_on_duplicates = duplicates[duplicates['is_duplicate'] == 1].shape[0] / duplicates.shape[0]
accuracy_on_differences = differences[differences['is_duplicate'] == 0].shape[0] / differences.shape[0]

In [ ]:
print("Точность на дубликатах: ", accuracy_on_duplicates)
duplicates[duplicates['is_duplicate'] == 0]

Точность на дубликатах:  0.975


,name_1,name_2,is_duplicate
pair_id,,,
196015,ÖZŞAHIN A.Ş,Ozsahin Suni Kosele Sanayii Ve Ticaret Anonim ...,0
453440,ORZEŁ S.A.,Orzel Spolka Akcyjna,0
30672,BINNE,Binné & Sohn GmbH & Co. KG Dachbaustoffwerk,0
197034,«ГК Полипласт»,ООО «Полипласт Новомосковск»,0
219409,kkpc,"KUMHO PETROCHEMICAL CO., LTD. (KOSE:A011780)",0
87463,NIS ( GPN SUBCIDIARY),NAFTNA INDUSTRIJA SRBIJE,0
168824,HEVEA B.V.,HeveaChem International,0
229597,Flex Sol,"Flex?Computing?(Suzhou) ?Co.,? Ltd.",0
290347,JSR CORPORATION (TSE:4185),BST ELASTOMERS CO.,0


In [ ]:
print("Точность на разных компаниях: ", accuracy_on_differences)
differences[differences['is_duplicate'] == 1]

Точность на разных компаниях:  0.641


,name_1,name_2,is_duplicate
pair_id,,,
141623,Corporacion Mexicana De Polimeros Sa De Cv,Riclan Sa,1
383967,Haris Trading,"Shenzhen Xinshidai Trading Co., Ltd.",1
90147,Cephas Medical Private Ltd.,Promod India Private Ltd.,1
47159,"Dongguan Zhaojing Trading Co., Ltd.",Epsilon Trading Llc,1
155291,Camelot Logistics,Panda Logistics Usa Inc. (New York),1
...,...,...,...
79243,Qingdao Ctr Logistics Co.. Ltd.,"Qingdao Changlong Stationery Co., Ltd.",1
337060,Kali Plast Private Ltd.,Ibm India Private Ltd.,1
47392,"Dongguan Gysun Electronic Plastics Co., Ltd.","Dongguan De Tai Electronics Co., Ltd.",1
